In [ ]:
from multiprocessing import Process, Queue
import rdkit
from rdkit.Chem import AllChem
import psi4
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
n2 = rdkit.Chem.MolFromSmiles("N#N")
AllChem.EmbedMolecule(n2)


print(rdkit.Chem.MolToXYZBlock(n2))

psi4.set_memory('4096 MB')
psi4.core.set_output_file('n2.txt', False)

n2_p4 = psi4.geometry(rdkit.Chem.MolToXYZBlock(n2))

# compute the electronic energy for N2
psi4.energy('B3LYP/6-311G**', molecule=n2_p4)


from multiprocessing import Process, Queue
q = Queue()
def perform_calculation(output_file:str, output_file_bool:bool,x:np.ndarray, calc:str, results_name:str, results_queue:list):
    psi4.core.set_output_file(output_file,output_file_bool)
    count = 1
    results = []
    for R in x:
        print(f'Performing calculation {count}/{len(x)}')
        count += 1
        n2_p4 = psi4.geometry(f"""
        N
        N 1 {R}
        """)
        results.append(psi4.energy(calc, molecule=n2_p4))
    results_queue.put({
        "data":results,
        "calculation":calc,
        "results_name":results_name,
        "x":x
    })
    return

def perform_plotting(all_results:dict):
    fig, ax = plt.subplots()
    b3lypegy_data = all_results['b3lypegy']['data']
    b3lypegy_label = all_results['b3lypegy']['calculation']
    b3lypegy_x = all_results['b3lypegy']['x']
    hfegy_data = all_results['hfegy']['data']
    hfegy_label = all_results['hfegy']['calculation']
    hfegy_x = all_results['hfegy']['x']
    b3lypegy2_data = all_results['b3lypegy2']['data']
    b3lypegy2_label = all_results['b3lypegy2']['calculation']
    b3lypegy2 = all_results['b3lypegy2']['x']
    ax.plot(b3lypegy_x, b3lypegy_data, marker='o', linestyle='solid', label=b3lypegy_label)
    ax.plot(hfegy_x, hfegy_data, marker='o', linestyle='solid', label=hfegy_label)
    ax.plot(b3lypegy2, b3lypegy2_data, marker='o', linestyle='solid', label=b3lypegy2_label)
    ax.set_xlabel(r'N-N Bond length ($\AA$)')
    ax.set_ylabel('Energy (Hartree)')
    ax.legend()
    fig.show()

calculation_kwargs = [
    {
        "output_file":"n2-pec-hf-321g.txt",
        "output_file_bool":False,
        "x":np.linspace(0.6, 5, 30),
        "calc":'HF/3-21G',
        "results_queue":q,
        "results_name":"hfegy"
    },
    {
        "output_file": 'n2-pec-b3lyp-321g.txt',
        "output_file_bool": False,
        "x": np.linspace(0.6, 5, 30),
        "calc": 'B3LYP/3-21G',
        "results_queue": q,
        "results_name":'b3lypegy'
    },
    {
        "output_file": 'n2-pec-b3lyp-631gs.txt',
        "output_file_bool": False,
        "x": np.linspace(0.6, 5, 30),
        "calc": 'B3LYP/3-21G',
        "results_queue": q,
        "results_name":'b3lypegy2'
    }
]


processes = []
for kwargs in calculation_kwargs:
    p = Process(target=perform_calculation, kwargs=kwargs)
    processes.append(p)
[p.start() for p in processes]
[p.join() for p in processes]


all_results = {}
while not q.empty():
    r = q.get()
    all_results[r['results_name']] = r

perform_plotting(all_results)


calc = 'B3LYP/6-31G*'
psi4.core.set_output_file('n2-opt-b3lyp-631gs.txt', False)

# pick a starting geometry (use rdkit value)
n2_geo = psi4.geometry(rdkit.Chem.MolToXYZBlock(n2))

# perform optimization with psi4.optimize
egy = psi4.optimize(calc, molecule=n2_geo)

# these functions will print things to the output file
n2_geo.print_distances()
n2_geo.print_bond_angles()

# this returns an acual matrix in units of Bohr
print(n2_geo.distance_matrix().to_array())

# convert to Angstrom
print(n2_geo.distance_matrix().to_array() * psi4.constants.bohr2angstroms)


psi4.core.set_output_file('n2-freq-b3lyp-631gs.txt', False)

# This version of psi4 has a bug! B3LYP frequency calculations don't work :(
psi4.frequency(calc, molecule=n2_geo)

calc = 'HF/6-31G*'
psi4.core.set_output_file('n2-opt-hf-631gs.txt', False)

# A frequency calculation must start from an optimized geometry at the same level of theory.
n2_geo = psi4.geometry(rdkit.Chem.MolToXYZBlock(n2))
egy = psi4.optimize(calc, molecule=n2_geo)

# Hartree-Fock frequency calculations do work
# Prof. Wang reported this problem as a bug, and it should be fixed in an upcoming release.
psi4.frequency(calc, molecule=n2_geo)

# Now let's look at a larger molecule, bonzoquinone
m1 = rdkit.Chem.MolFromSmiles('C1=CC(=O)C=CC1=O')
m1

# add H atoms, embed, visualize
m1 = rdkit.Chem.AddHs(m1)
AllChem.EmbedMolecule(m1)
m1

import nglview

nglview.show_rdkit(m1)

bq = psi4.geometry(rdkit.Chem.MolToXYZBlock(m1))
psi4.core.set_output_file('bq-egy-b3lyp-631gs.txt')
calc = 'B3LYP/6-31G*'
egy = psi4.energy(calc, molecule=bq)
egy

psi4.core.set_output_file('bq-opt-b3lyp-631gs.txt')
calc = 'B3LYP/6-31G*'
optegy = psi4.optimize(calc, molecule=bq)
optegy

print(abs(egy - optegy) * psi4.constants.hartree2kcalmol)

# get optimized structure into rdkit... not as straightforward as it might seem!
# the xyz coordinates don't contain bonding information, which rdkit requires to construct the molecular graph
# however, as long as we initialize the psi4 calculation with an rdkit XYZBlock
# we can simply replace the coordinates with the optimized ones, because the atoms will be in the same order

# get geometry, and reshape it into an Nx3 array
psi4_geom = bq.to_dict()['geom'].reshape(-1, 3)

from rdkit.Chem import rdchem

# create a new version of the molecule
m2 = rdkit.Chem.MolFromSmiles('C1=CC(=O)C=CC1=O')
m2 = rdkit.Chem.AddHs(m2)
AllChem.EmbedMolecule(m2)
conf = m2.GetConformer(0)
for i in range(m2.GetNumAtoms()):
    conf.SetAtomPosition(i, psi4_geom[i])

# add this as a second conformer to the molecule, then remove the original
m2.AddConformer(conf, assignId=True)
m2.RemoveConformer(0)
nglview.show_rdkit(m2)

# Compute RMSD of the optimized structure with the conformer generated by rdkit
rdkit.Chem.rdMolAlign.AlignMol(m1, m2)

# Now let's try to calculate what the energy of this structure would be in water
# This is done using a polarizable continuum model -- approximating
# a solvation environment by surrounding the molecule with charges that resemble
# the charge distribution of water molecules, but not using a quantum mechanical treatment
# of water molecules themselves.

# https://pubs.acs.org/doi/abs/10.1021/cr9904009

# Set implicit solvent options. Due to a quirk, this cell
# may only be called once. If you want to use a different solvent, you have to restart the kernel!
# The details of the "Cavity" are fairly standard values that should only be
# adjusted if you have more expertise
pcm_string = """
Units = Angstrom
Medium {
  SolverType = IEFPCM
  Solvent = Water
}
Cavity {
  RadiiSet = UFF
  Type = GePol
  Scaling = False
  Area = 0.3
  Mode = Implicit
}
"""
psi4.pcm_helper(pcm_string)
psi4.set_options({'pcm': True, 'pcm_scf_type': 'total'})

# Compute implicit solvent energy.
psi4.core.set_output_file('bq-solvation-b3lyp-631gs.txt', False)
solvegy = psi4.energy('b3lyp/6-31g*', molecule=bq)

# The energy of solvation is approximately the difference between the gas-phase energy and the solvated energy
# Ideally one would optimize the molecule's geometry inside the solvent, but that is very challenging as
# a standard method is not readily available. It is possible to do, but the details are well beyond the scope
# of this course
print((solvegy - optegy) * psi4.constants.hartree2kcalmol)

